In [ ]:
import numpy as np
import plotly.express as px
import pandas as pd

## part 1

In [ ]:
def grad_n(r: np.array) -> np.array:
    grad_n = -2 / (sum(r**2) ** (3/2)) * r
    return grad_n

def n(r: np.array) -> float:
    n_ = 1 + 1 / np.linalg.norm(r)
    return n_


def step(r: np.array, v: np.array, s: np.array, l: np.array, i:int):
    # n_, grad_n = n(r)

    r[i+1] = r[i] + v[i] * s[i]
    v[i+1] = (grad_n(r[i]) * s[i] + n(r[i]) * v[i]) / n(r[i+1])
    s[i+1] =  np.linalg.norm(r[i+1] - r[i])

    l[i] = np.linalg.norm(r[i+1] - r[i]) / s[i]
    return r, v, s , l

def cut_all_zeros(n, *args):
    clean_args = []
    for arg in args:
        clean_args.append(args[:n])
    return tuple(clean_args)

def solve(x0, z0, alpha0, s0, epochs, z_stop_condition=None, display=False):
    vx0 = np.sin(alpha0)
    vz0 = -np.cos(alpha0)

    r = np.zeros((epochs, 2))
    v = np.zeros((epochs, 2))
    s = np.zeros(epochs)
    l = np.zeros(epochs)

    r[0] = [x0, z0]
    v[0] = [vx0, vz0]
    s[0] = s0
    s[1] = s0

    for i in range(epochs -1):
        r, v, s, l = step(r, v, s, l, i)

        if z_stop_condition != None and r[i+1, 1] < z_stop_condition:
            print(f'Epoch: {i}')
            r = r[:i+1]
            v = v[:i+1]
            s = s[:i+1]
            break
    
    if display:
        fig = px.line(x=r[:,0], y=r[:,1], labels={'x': 'numeric'})

        thory_line = np.array([
            r[0],
            r[0] + v[0] * 1000,
        ])
        fig.add_scatter(x=thory_line[:,0], y=thory_line[:,1], name="Theory")
        fig.show()
        
    return r, v, s

In [ ]:
# The been curve for 2 deg
# I define the z_stop_condition to stop the simulation when the been is fully turnerd
x0 = 0
z0 = 1000
alpha0 = np.deg2rad(1) # deg 
vx0 = np.sin(alpha0)
vz0 = -np.cos(alpha0)
s0 = 1
epochs = 1000

r, v, s = solve(x0, z0, alpha0, s0, epochs, z_stop_condition=0, display=False)
fig = px.line(x=r[:,0], y=r[:,1], labels={'x': 'numeric'})

thory_line = np.array([
    r[0],
    r[0] + v[0] * 1000,
])

fig.add_scatter(x=thory_line[:,0], y=thory_line[:,1], name="Theory", )

fig.update_layout({
    'xaxis_title': 'x',
    'yaxis_title': 'z',
    'title': 'Been curve for 2 deg',
    # 'legend_title': 'Legend Title',
    'font': {
        'size': 16,
    },
    'width': 800  ,
    'height': 600,
    # 'showlegend': False,
    # 'grid': {}
        
    # 'template': 'plotly_white'
})

fig.show()

# fig.write_image("fructose.eps")

### B

In [ ]:
# from thory calculation
def calculate_angle_shift(alpha0):
    vx0 = np.sin(alpha0)
    vz0 = -np.cos(alpha0)
    
    b = -z0 * vx0/ vz0 
    print(f'deg = {alpha0} b: {b}')
    

    r, v, s = solve(x0=0, z0=1000, alpha0=alpha0, s0=1, epochs=1000, z_stop_condition=0, display=True)
    delta_alpha = np.rad2deg(np.arccos(np.dot(v[0], v[-1]) / (np.linalg.norm(v[0]) * np.linalg.norm(v[-1]))))
    delta_alpha_theory = 2 / b
    error = np.abs(delta_alpha - delta_alpha_theory) / delta_alpha
    print(f'delta_alpha: {delta_alpha}')
    print(f'delta_alpha_theory: {delta_alpha_theory}')
    return error

angles = np.arange(0.01, 0.1, 0.01)
errors = []
for angle in angles:
    errors.append(100 * calculate_angle_shift(np.deg2rad(angle)))

fig = px.scatter(x=angles, y=errors) 
fig.update_layout({
    'xaxis_title': 'angle [deg]',
    'yaxis_title': 'relative error [%]',
    # 'title': 'Been curve for 2 deg',
    # 'legend_title': 'Legend Title',
    'font': {
        'size': 16,
    },
    'width': 800  ,
    'height': 600,
    # 'showlegend': False,
    # 'grid': {}
        
    # 'template': 'plotly_white'
})

fig.show()

fig.write_image("part1_b.eps")

## part 2